In [1]:
from google.colab import files
upload = files.upload()

Saving pubmedqa_new.csv to pubmedqa_new (2).csv


In [2]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz (120.2 MB)
  Preparing metadata (setup.py) ... done


In [3]:
# !pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
import scispacy
import en_ner_bc5cdr_md
nlp = spacy.load('en_ner_bc5cdr_md')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
# df = pd.read_csv("data/pubmedqa_new.csv",encoding='latin-1')
df = pd.read_csv("pubmedqa_new.csv",encoding='latin-1')
df

,Context,Question,Answer
0,['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the pl...,Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?,"Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best..."
1,['Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle....,Landolt C and snellen e acuity: differences in strabismus amblyopia?,"Using the charts described, there was only a slight overestimation of visual acuity by the Snellen E compared to the Landolt C, even in strabismus amblyopia. Small differences in the lower visual ..."
2,"['Apparent life-threatening events in infants are a difficult and frequent problem in pediatric practice. The prognosis is uncertain because of risk of sudden infant death syndrome.', 'Eight infan...","Syncope during bathing in infants, a pediatric form of water-induced urticaria?","""Aquagenic maladies"" could be a pediatric form of the aquagenic urticaria."
3,"['The transanal endorectal pull-through (TERPT) is becoming the most popular procedure in the treatment of Hirschsprung disease (HD), but overstretching of the anal sphincters remains a critical i...",Are the long-term results of the transanal pull-through equal to those of the transabdominal pull-through?,Our long-term study showed significantly better (2-fold) results regarding the continence score for the abdominal approach compared with the transanal pull-through. The stool pattern and enterocol...
4,"['Telephone counseling and tailored print communications have emerged as promising methods for promoting mammography screening. However, there has been little research testing, within the same ran...",Can tailored interventions increase mammography use among HMO women?,"The effects of the intervention were most pronounced after the first intervention. Compared to usual care, telephone counseling seemed particularly effective at promoting change among nonadherent ..."
...,...,...,...
995,"['After 34 weeks gestation, summary measures of location for birthweight (e.g means and centiles) increase more slowly for Australian Aborigines than for whites. A similar pattern has been observe...",Does gestational age misclassification explain the difference in birthweights for Australian aborigines and whites?,Gestational age misclassification is an unlikely explanation for the reported divergence in average birth-weights for Aborigines and whites. The results might help with the interpretation of other...
996,"['To evaluate the accuracy of ultrasonographic examination in boys with an undescended testis.', 'All patients who were referred to the paediatric surgeon after detection of an undescended testis ...",Is there any interest to perform ultrasonography in boys with undescended testis?,Sonography has no place in the diagnosis of undescended testis.
997,['We analyzed the pharmacokinetic-pharmacodynamic relationship of vancomycin to determine the drug exposure parameters that correlate with the efficacy and nephrotoxicity of vancomycin in patients...,Is peak concentration needed in therapeutic drug monitoring of vancomycin?,These results suggest little need to use peak concentration in vancomycin TDM because Cmin/MIC and Cmin are sufficient to predict the efficacy and safety of vancomycin.
998,"['This investigation assesses the effect of platelet-rich plasma (PRP) gel on postoperative pain, swelling, and trismus as well as healing and bone regeneration potential on mandibular third molar...",Can autologous platelet-rich plasma gel enhance healing after surgical extraction of mandibular third molars?,"T

In [5]:
for doc in df["Context"]:
    doc = nlp(doc)
    for tok in doc:
      print(tok.text, "...", tok.dep_)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
questions ... dobj
to ... case
a ... det
health ... compound
care ... compound
professional ... nmod
was ... cop
of ... case
interest ... compound
to ... case
202 ... nmod
( ... punct
70.4 ... nummod
% ... appos
) ... punct
participants ... ROOT
. ... punct
" ... punct
] ... punct
[ ... punct
' ... punct
To ... mark
assess ... advcl
whether ... mark
eligibility ... nsubj
to ... case
an ... det
adjuvant ... amod
chemotherapy ... compound
protocol ... nmod
in ... case
itself ... nmod
represents ... ccomp
a ... det
good ... amod
prognostic ... amod
factor ... dobj
after ... case
radical ... amod
cystectomy ... nmod
for ... case
bladder ... compound
cancer ... nmod
. ... punct
' ... punct
, ... punct
' ... punct
Between ... case
April ... nmod
1984 ... nummod
and ... cc
May ... conj
1989 ... nummod
, ... punct
our ... nmod:poss
institution ... nsubj
entered ... ccomp
35 ... nummod
patients ... dobj
with ... case


In [6]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [7]:
entity_pairs = []

for i in tqdm(df["Context"]):
  entity_pairs.append(get_entities(i))

100%|██████████| 1000/1000 [00:49<00:00, 20.12it/s]


In [8]:
entity_pairs

[['controls  that', 'mitochondrial  dynamics'],
 ['groups strabismus results', 'visual ETDRS acuity'],
 ['year follow-up year three', 'increase'],
 ['age', 'long-term scoring outcome'],
 ['women print who', 'tailored  print'],
 ['findings', 'bleeding  sources'],
 ['1-year  mortality', 'emergency laparotomy'],
 ['probit model CHOPIT model', 'two  changes'],
 ['age-related  cIMT', 'further  relationship'],
 ['stay ward stay This', '23 stay hours'],
 ['period police traffic enforcement', 'traffic fatalities'],
 ['complications  Patients', 'FH'],
 ['fibrosis  Differences', '0.00049'],
 ['significant family change', 'ordered family tests/referrals'],
 ['Significant EUS differences', '= EUS .04'],
 ['mode', 'application  relief'],
 ['LRT group Serum values', 'Kaplan-Meier  method'],
 ['Fearful influenza Uninformeds', 'ever influenza vaccine'],
 ['Experimental learning curve', 'whole porcine space'],
 ['risk factors regression dilution ratio', '0.013'],
 ['79.2 ED discharges casualties', 'med

In [9]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", [pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [10]:
relations = [get_relation(i) for i in tqdm(df['Context'])]

100%|██████████| 1000/1000 [00:50<00:00, 19.66it/s]


In [11]:
pd.Series(relations).value_counts()

was                 41
found               41
showed              35
associated          32
had                 32
                    ..
lack                 1
highest              1
discarded            1
investigate main     1
revealed high        1
Length: 440, dtype: int64

In [12]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})
kg_df

,source,target,edge
0,controls that,mitochondrial dynamics,resulted
1,groups strabismus results,visual ETDRS acuity,similar
2,year follow-up year three,increase,continue
3,age,long-term scoring outcome,noted
4,women print who,tailored print,effective
...,...,...,...
995,Extreme types,misclassification divergence,required
996,other structures,abdominal testes,missed
997,characteristic curves,predictive values,revealed high
998,PRP group PRP difference,35 years,significant


In [13]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [14]:
# !pip install networkx==2.8.6

In [18]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [19]:
G_determine=nx.from_pandas_edgelist(kg_df[kg_df['edge']=="determine"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G_determine, k = 0.5) # k regulates the distance between nodes
nx.draw(G_determine, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()